In [1]:
import re
import google.auth
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request

In [2]:
# Authenticate in Google Colab
auth.authenticate_user()
creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)

In [3]:
def create_google_doc(title, content):
    document = docs_service.documents().create(body={"title": title}).execute()
    doc_id = document['documentId']
    update_doc_content(doc_id, content)
    return f"https://docs.google.com/document/d/{doc_id}"

def get_document_length(doc_id):
    """Fetch the current length of the Google Doc."""
    document = docs_service.documents().get(documentId=doc_id).execute()
    return document['body']['content'][-1]['endIndex'] - 1

In [14]:
def update_doc_content(doc_id, content):
    requests = []
    index = 1  # Track position for inserting text

    for line in content.split('\n'):
        if line.startswith('# '):
            text = line.replace('# ', '') + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(text), "endIndex": index},
                    "paragraphStyle": {"namedStyleType": "HEADING_1"},
                    "fields": "namedStyleType"
                }
            })

        elif line.startswith('## '):
            text = line.replace('## ', '') + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(text), "endIndex": index},
                    "paragraphStyle": {"namedStyleType": "HEADING_2"},
                    "fields": "namedStyleType"
                }
            })

        elif line.startswith('### '):
            text = line.replace('### ', '') + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": index - len(text), "endIndex": index},
                    "paragraphStyle": {"namedStyleType": "HEADING_3"},
                    "fields": "namedStyleType"
                }
            })

        elif re.match(r'- \[ \] @\w+', line):
            # Replace markdown checkbox with Unicode checkbox
            text = '☐ ' + line.replace('- [ ]', '').strip() + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)

            # Bold the mention (@name)
            mention_match = re.search(r'(@\w+)', text)
            if mention_match:
                mention_start = index - len(text) + text.find(mention_match.group(0))
                mention_end = mention_start + len(mention_match.group(0))
                requests.append({
                    "updateTextStyle": {
                        "range": {
                            "startIndex": mention_start,
                            "endIndex": mention_end
                        },
                        "textStyle": {
                            "bold": True
                        },
                        "fields": "bold"
                    }
                })

        elif re.match(r'- @\w+', line):
            text = line + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": index - len(text), "endIndex": index},
                    "textStyle": {"bold": True},
                    "fields": "bold"
                }
            })

        else:
            text = line + '\n'
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text
                }
            })
            index += len(text)

    # Execute the requests in batch
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()


In [15]:
markdown_content = '''# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
  * Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
  * Reduced load time by 40%
  * Implemented caching solution
  * Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration


### 2. Current Challenges
  * Resource constraints in QA team
  * Third-party API integration delays
  * User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
  * Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
  * Technical Debt
  * Code refactoring
  * Documentation updates



## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''

In [16]:
doc_url = create_google_doc("Meeting Notes", markdown_content)

In [17]:
print(f"Google Doc created: {doc_url}")

Google Doc created: https://docs.google.com/document/d/1fL1Fg-8jb6Ix5JZLYiulIYcfHhUNnAwhddvmNsRO71U
